In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [7]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("./speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [8]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#DataFlair - Train the model
model.fit(x_train,y_train)

(576, 192)
Features extracted: 180


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [9]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.75%


In [11]:
test_audio = "./speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav"
# file = glob.glob(test_audio)
feature=extract_feature(test_audio, mfcc=True, chroma=True, mel=True).reshape(1,-1)
print(feature)

[[-6.93497009e+02  5.00643921e+01  5.71450591e-01  1.43299656e+01
   3.33636999e+00 -2.54071975e+00 -4.05790949e+00 -1.07119999e+01
  -7.29413939e+00  1.74018872e+00 -4.19064283e+00  1.95466173e+00
  -5.24789381e+00  2.78142977e+00 -3.16756773e+00 -3.40008307e+00
  -2.37803221e+00 -5.68717539e-01 -6.47753334e+00 -1.24320555e+00
  -2.80542517e+00 -5.43635798e+00 -4.46875393e-01 -3.63516617e+00
  -2.98372650e+00 -5.63902617e-01 -1.65101993e+00 -5.55944860e-01
  -3.41018438e+00 -2.24465466e+00 -3.13058877e+00 -2.70089960e+00
  -1.88821328e+00 -5.54154277e-01 -3.96459913e+00 -2.13485193e+00
  -3.94577074e+00 -1.62457860e+00 -2.03990722e+00 -3.62910867e+00
   6.33734703e-01  6.48760676e-01  6.21744275e-01  6.34553194e-01
   6.60345614e-01  6.60463989e-01  7.00949013e-01  7.31556356e-01
   7.46921539e-01  7.23420441e-01  7.16119647e-01  6.30602837e-01
   6.52830636e-07  2.22643266e-05  8.42613052e-04  1.37719270e-02
   2.83742324e-02  3.12111359e-02  8.02663807e-03  3.56961833e-03
   2.32551

In [12]:
pred = model.predict(feature)
print(pred)

['happy']


In [14]:
import subprocess
subprocess.call(['ffmpeg', '-i', 'dancin.mp3',
                   'result.wav'])

FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'